# Daynamisches Labyrinth
#### _Sucherverfahren: A*_
##### __Aufgabenstellung:__
Es gilt den Weg durch ein labyrinthartiges Gebiet zu finden. Die Landschaft
verändert sich mit jedem Schritt durch die Aktion. Die Landschaft besteht aus
20 Feldern, die in 4 Zeilen und 5 Spalten angeordnet sind. Ein Spielzug kann
entweder einen Spielstein an einer mit einem blauen Pfeil markierten Stelle in
das Spielfeld schieben oder die Spielfigur beliebig weit entlang eines begehbaren Wegs ziehen. Wird ein Spielstein seitlich in das Feld geschoben, so
verschiebt sich die ganze Zeile und der Stein auf den gegenüberliegenden
Seite verlässt das Spielfeld (und kann beim nächsten Zug genutzt werden).
Sollte sich die Spielfigur auf dem herausgeschobenen Feld befinden, wird sie
auf das hineingeschobene Feld gesetzt. Ziel ist es, mit einer möglichst geringen Zahl an Zügen von der Eingangsposition zur Ausgangsposition (rote Pfeile) zu kommen. Der Ein- und Ausgang muss beim Durchschreiten begehbar
sein.

## Ansatz:
Das 

In [1]:
import csv
from typing import List


In [2]:
class Token:
    def __init__(self, left: bool, right: bool, up: bool, down: bool):
        self.canReach = {"left": left, "right": right, "up": up, "down": down}

    def __str__(self):
        row1 = ""
        if self.canReach["up"]:
            row1 = "   |   "
        row2 = ""
        if self.canReach["left"]:
            row2 += "-- "
        else:
            row2 += "   "
        row2 += "*"
        if self.canReach["right"]:
            row2 += " --"
        else:
            row2 += "   "

        row3 = ""
        if self.canReach["down"]:
            row3 = "   |   "
        return row1 + "\n" + row2 + "\n" + row3

    def str_row1(self):
        if self.canReach["up"]:
            return "   |   "
        else:
            return "       "

    def str_row2(self, is_start: bool = False, is_goal: bool = False, is_player: bool = False):
        row2 = ""
        if self.canReach["left"]:
            row2 += "-- "
        else:
            row2 += "   "
        if is_player:
            row2 += "*"
        elif is_goal:
            row2 += "g"
        elif is_start:
            row2 += "s"
        else:
            row2 += "*"
        if self.canReach["right"]:
            row2 += " --"
        else:
            row2 += "   "
        return row2

    def str_row3(self):
        if self.canReach["down"]:
            return "   |   "
        else:
            return "       "


## Gewählten Model
Für die Umsetztung des A* Suchverfahren im Anwednungsbeispiel eines dynamischen Labyrinths, wurden folgende Models erstellt:
- Node
- Edge
- NodeSet
- TokenPush

## Node
Eine Node ist dabei ein Knotenpunkt im Labyrint. Auf jedem Knotenpunk im Labyrint liegt ein Token, also ein Spielstein. Die Notes haben dabei immer eine feste Postion, bei einer Verschiebung ändern sich nur die Tokens, die auf einem Node liegen. Die Anzahl der einzelnen Nodes wird ist abhängig von der Anzahl der Felder eines Labyrinths. Die Testdaten beeinhalten 20 Felder, die in vier Zeilen und fünf Spalten aufgeteilt sind.
#### Attribute der Node
Eine Node hat mehrere Eigenschaftem, die bei ihrer Initalisierung gesetzt werden. Als erstes wird festgelegt, welcher Token sich aktuell auf dem Node befindet. Diese Information kann sich bei einer Verschiebung der Tokens ändern.
Unter den Attributen row und col sind die Koordinaten einer Node gespeichert, diese werden nach der Initalisierung einer Node nichtmehr verändert. Es hat dabei Fünf Reihen, welche von Null bis Vier nummeriert sind. Die oberste Reihe ist die Nummer Null und die unterste hat die Nummer Vier. Bei den Spalten, gibt es nur Viert, auch diese fangen mit Null and und gehen bis zu Drei. Hierbei beginnt die Nummerierung von Links nach Rechts. <br>
Das Attribut "isGoal" definiert, ob es sich bei der Node schon um das Zielnode handelt. <br>
Edges <br>
token_pushes_calculated <br>
path_in_edge <br>
stepsToReach <br>
parent_set <br>

In [3]:
class Node:
    def __init__(self, token, row, col, is_goal, parent_set=None):
        self.token = token
        self.row = row
        self.col = col
        self.isGoal = is_goal
        self.edges = []
        self.token_pushes_calculated = False
        self.path_in_edge = None
        self.stepsToReach = sys.maxsize
        self.parent_set = parent_set

    def __str__(self):
        return "S" + str(self.parent_set.id) + " R" + str(self.row) \
               + " C" + str(self.col) + " $" + str(self.get_costs())

    def __lt__(self, other):
        return self.get_costs() < other.get_costs()

    def get_costs(self):
        goal_row = self.parent_set.dest_row
        goal_col = self.parent_set.dest_col
        row_distance = self.row - goal_row
        col_distance = self.col - goal_col
        abs_distance = abs(row_distance) + abs(col_distance)
        return self.stepsToReach + abs_distance

    def update_steps_to_reach(self, new_steps_to_reach, edge):
        if new_steps_to_reach < self.stepsToReach:
            self.path_in_edge = edge
            self.stepsToReach = new_steps_to_reach

    def get_edges(self):
        if self.token_pushes_calculated:
            return self.edges
        free_token = self.parent_set.free_token
        token_push_edges = []
        for i in range(1, 5):
            token_push = TokenPush(free_token, i, self.parent_set, self)
            edge = Edge(self, token_push.activeNodeAfter, token_push)
            token_push_edges.append(edge)
        self.edges.extend(token_push_edges)
        self.token_pushes_calculated = True
        return self.edges

    def set_parent_set(self, parent_set):
        self.parent_set = parent_set

    def get_is_valid_goal(self):
        if self.isGoal:
            print("goal node reached")
        return self.isGoal and self.token.canReach["up"]

    def get_token(self):
        if self.token is not None:
            return self.token
        else:
            print("Empty Token on node")
            return Token(False, False, False, False)


class Edge:
    def __init__(self, source: Node, dest: Node, movement) -> None:
        self.source = source
        self.dest = dest
        self.movement = movement

    def __str__(self):
        return str(self.movement) + \
               " from R" + str(self.source.row) + \
               " C" + str(self.source.col) + " to R" + \
               str(self.dest.row) + " C" + str(self.dest.col)


class NodeSet:
    total_nodesets = 0

    def __init__(self, nodes: List[List[Node]], free_token: Token, dest_row: int, dest_col: int):
        self.nodes = nodes
        for row in range(len(nodes)):
            for col in range(len(nodes[row])):
                node = nodes[row][col]
                node.set_parent_set(self)
                if col > 0:
                    left_node = nodes[row][col - 1]
                    if node.get_token().canReach["left"] and left_node.get_token().canReach["right"]:
                        node.edges.append(Edge(node, left_node, "LEFT"))
                if col < len(nodes[row]) - 1:
                    right_node = nodes[row][col + 1]
                    if node.get_token().canReach["right"] and right_node.get_token().canReach["left"]:
                        node.edges.append(Edge(node, right_node, "RIGHT"))
                if row > 0:
                    up_node = nodes[row - 1][col]
                    if node.get_token().canReach["up"] and up_node.get_token().canReach["down"]:
                        node.edges.append(Edge(node, up_node, "UP"))
                if row < len(nodes) - 1:
                    down_node = nodes[row + 1][col]
                    if node.get_token().canReach["down"] and down_node.get_token().canReach["up"]:
                        node.edges.append(Edge(node, down_node, "DOWN"))
        self.free_token = free_token
        self.dest_row = dest_row
        self.dest_col = dest_col
        self.id = NodeSet.total_nodesets
        NodeSet.total_nodesets += 1

    def __str__(self):
        result = ""
        for row in self.nodes:
            row1, row2, row3 = "", "", ""
            for node in row:
                row1 += node.get_token().str_row1()
                row2 += node.get_token().str_row2(is_goal=node.isGoal)
                row3 += node.get_token().str_row3()
            result += row1 + "\n" + row2 + "\n" + row3 + "\n"
        return result

    def __eq__(self, other):
        return self.id == other.id


def get_nodelist_deepcopy(nodelist: List[Node]):
    list_copy = []
    for node in nodelist:
        list_copy.append(Node(node.token, node.row, node.col, node.isGoal))
    return list_copy


def calc_new_set(old_set: NodeSet, pushed_token: Token, row: int):
    new_nodes: List[List[Node]] = []
    freed_token: Token = None
    for row_index in range(len(old_set.nodes)):
        row_deepcopy = get_nodelist_deepcopy(old_set.nodes[row_index])
        if row_index == row and row_index % 2 == 0:
            freed_token = push_to_right(row_deepcopy, pushed_token)
        elif row_index == row and row_index % 2 == 1:
            freed_token = push_to_left(row_deepcopy, pushed_token)
        new_nodes.append(row_deepcopy)
    return NodeSet(new_nodes, freed_token, old_set.dest_row, old_set.dest_col)


def push_to_left(nodes: List[Node], token: Token):
    freed_token = nodes[0].get_token()
    for i in range(len(nodes) - 1):
        nodes[i].token = nodes[i + 1].get_token()
    nodes[-1].token = token
    return freed_token


def push_to_right(nodes, token):
    freed_token = nodes[-1].get_token()
    for i in range(1, len(nodes)):
        index = len(nodes) - i
        nodes[index].token = nodes[index - 1].token
    nodes[0].token = token
    return freed_token


def calc_active_node_after(new_node_set, row, active_node_before):
    new_col = active_node_before.col
    if row == active_node_before.row and row % 2 == 0:
        new_col = active_node_before.col + 1
        if new_col > len(new_node_set.nodes[0]) - 1:
            new_col = 0
    elif row == active_node_before.row and row % 2 == 1:
        new_col = active_node_before.col - 1
        if new_col < 0:
            new_col = len(new_node_set.nodes[0]) - 1
    return new_node_set.nodes[active_node_before.row][new_col]


class TokenPush:
    def __init__(self,
                 pushed_token: Token,
                 row: int,
                 old_set: NodeSet,
                 active_node_before: Node):
        self.pushedToken = pushed_token
        self.row = row
        self.oldSet = old_set
        self.activeNodeBefore = active_node_before
        self.newSet = calc_new_set(old_set, pushed_token, row)
        self.freedToken = self.newSet.free_token
        self.activeNodeAfter = calc_active_node_after(self.newSet, row, active_node_before)

    def __str__(self):
        return "PUSH TOKEN in Row " + str(self.row)


In [4]:
class PriorityQueue:
    def __init__(self):
        self.items = []

    def push(self, item):
        self.items.append(item)

    def pop(self):
        if self.is_empty():
            return None
        self.items.sort()
        to_return = self.items[0]
        self.items.remove(to_return)
        return to_return

    def __contains__(self, item):
        return item in self.items

    def __str__(self):
        string = "[ "
        for item in self.items:
            string += " " + str(item) + " "
        string += "]"
        return string

    def is_empty(self):
        return len(self.items) < 1

In [5]:
def a_star(start_node: Node):
    open_nodes = PriorityQueue()
    closed_nodes = []
    open_nodes.push(start_node)
    start_node.stepsToReach = 0
    while not open_nodes.is_empty():
        current_node: Node = open_nodes.pop()
        if current_node.get_is_valid_goal():
            return current_node
        closed_nodes.append(current_node)
        expand_node(current_node, open_nodes, closed_nodes)
    print("Something went wrong, could not find path")
    return None


def expand_node(current_node, open_nodes, closed_nodes):
    out_edges = current_node.get_edges()
    successor_nodes: List[Node] = [edge.dest for edge in out_edges]
    for i in range(len(successor_nodes)):
        successor_node = successor_nodes[i]
        if successor_node in closed_nodes:
            continue
        tentative_g = current_node.stepsToReach + 1
        if successor_node in open_nodes and tentative_g >= successor_node.stepsToReach:
            continue
        successor_node.update_steps_to_reach(tentative_g, out_edges[i])
        if successor_node not in open_nodes:
            open_nodes.push(successor_node)

In [6]:
def a_star(start_node: Node):
    open_nodes = PriorityQueue()
    closed_nodes = []
    open_nodes.push(start_node)
    start_node.stepsToReach = 0
    while not open_nodes.is_empty():
        current_node: Node = open_nodes.pop()
        if current_node.get_is_valid_goal():
            return current_node
        closed_nodes.append(current_node)
        expand_node(current_node, open_nodes, closed_nodes)
    print("Something went wrong, could not find path")
    return None


def expand_node(current_node, open_nodes, closed_nodes):
    out_edges = current_node.get_edges()
    successor_nodes: List[Node] = [edge.dest for edge in out_edges]
    for i in range(len(successor_nodes)):
        successor_node = successor_nodes[i]
        if successor_node in closed_nodes:
            continue
        tentative_g = current_node.stepsToReach + 1
        if successor_node in open_nodes and tentative_g >= successor_node.stepsToReach:
            continue
        successor_node.update_steps_to_reach(tentative_g, out_edges[i])
        if successor_node not in open_nodes:
            open_nodes.push(successor_node)

In [7]:
GOAL_COL = 3
GOAL_ROW = 0

tokenDict = {
    "0": Token(False, True, False, True),
    "1": Token(True, False, False, True),
    "2": Token(False, True, True, False),
    "3": Token(True, False, True, False),
    "4": Token(True, True, False, True),
    "5": Token(True, False, True, True),
    "6": Token(True, True, True, False),
    "7": Token(False, True, True, True),
    "8": Token(False, False, True, True),
    "9": Token(True, True, False, False)
}


def get_nodeset_and_free_token_from_file(filepath):
    nodes: List[List[Node]] = []
    free_token: Token = Token(False, False, False, False)
    with open(filepath) as csvFile:
        csv_reader = csv.reader(csvFile, delimiter=";")
        row_index = 0
        for row in csv_reader:
            if row_index < 5:
                nodes.append([])
                item_index = 0
                for item in row:
                    is_goal = row_index == GOAL_ROW and item_index == GOAL_COL
                    token = tokenDict[item]
                    nodes[row_index].append(Node(token, row_index, item_index, is_goal))
                    item_index += 1
            else:
                free_token = tokenDict[str(row[0])]
            row_index += 1
    return NodeSet(nodes, free_token, GOAL_ROW, GOAL_COL)


In [8]:
INPUT_FILE_PATH = "Puzzle_6.CSV"
START_NODE_ROW = 4
START_NODE_COL = 1

GOAL_NODE_ROW = 0
GOAL_NODE_COL = 2


def print_path_edges(start_node, end_node):
    node = end_node
    reversed_edges = []
    while node.path_in_edge is not None:
        reversed_edges.append(node.path_in_edge)
        node = node.path_in_edge.source
    edges = reversed(reversed_edges)
    for edge in edges:
        print(edge)
        if not type(edge.movement) == type("  "):
            print(edge.dest.parent_set)
            print("new free token:")
            print(edge.dest.parent_set.free_token)


initial_nodeset = get_nodeset_and_free_token_from_file(INPUT_FILE_PATH)

print("Read Input:")
print(str(initial_nodeset))
print("Free Token:")
print(str(initial_nodeset.free_token))
first_node = initial_nodeset.nodes[START_NODE_ROW][START_NODE_COL]
final_node = a_star(first_node)
print_path_edges(first_node, final_node)

Read Input:
   |             |      |   
   * ---- * ---- * --   g --
          |             |   
          |      |          
-- *   -- *   -- *   -- * --
   |      |             |   
                 |      |   
-- * --   * --   * ---- *   
          |                 
   |             |          
-- * ---- * ---- *   -- *   
          |      |      |   
          |      |          
   * --   *   -- * ---- * --
   |      |                 

Free Token:
   |   
   * --
   |   
goal node reached
UP from R4 C1 to R3 C1
PUSH TOKEN in Row 2 from R3 C1 to R3 C1
   |             |      |   
   * ---- * ---- * --   g --
          |             |   
          |      |          
-- *   -- *   -- *   -- * --
   |      |             |   
   |                    |   
   * ---- * --   * --   * --
   |             |          
   |             |          
-- * ---- * ---- *   -- *   
          |      |      |   
          |      |          
   * --   *   -- * ---- * --
   |      |                 
